In [2]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# 1. The Dynamic Hardware Check
# This will hunt for your local NVIDIA GPU. If it fails, it safely falls back to CPU.
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

print(f"🖥️ Hardware Detected: Running on {device.type.upper()}")

# 2. Download Base Architecture (Only takes a moment locally)
print("\nLoading the base ClinicalBERT architecture...")
tokenizer = AutoTokenizer.from_pretrained("emilyalsentzer/Bio_ClinicalBERT")
model = AutoModelForSequenceClassification.from_pretrained(
    "emilyalsentzer/Bio_ClinicalBERT",
    num_labels=49, 
    problem_type="multi_label_classification"
)

# 3. Load Your Trained Weights
# Update this path if you placed the .pt file directly in the main folder instead!
model_path = 'saved_models/heal_bridge_best_model.pt'

print(f"\nInjecting custom weights from: {model_path}...")
# map_location=device ensures the weights load correctly regardless of CPU/GPU
checkpoint = torch.load(model_path, map_location=device)
model.load_state_dict(checkpoint['model_state_dict'])

# 4. Push model to the detected hardware and lock it
model.to(device)
model.eval()

print("\n✅ SUCCESS: Heal Bridge AI is locked, loaded, and ready for deployment!")

🖥️ Hardware Detected: Running on CPU

Loading the base ClinicalBERT architecture...


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at emilyalsentzer/Bio_ClinicalBERT and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



Injecting custom weights from: saved_models/heal_bridge_best_model.pt...

✅ SUCCESS: Heal Bridge AI is locked, loaded, and ready for deployment!


In [3]:
# The Exact Disease List (Crucial for mapping the 49 neurons back to English)
DISEASES = [
    'Acute COPD exacerbation / infection', 'Acute dystonic reactions', 'Acute laryngitis', 
    'Acute otitis media', 'Acute pulmonary edema', 'Acute rhinosinusitis', 'Allergic sinusitis', 
    'Anaphylaxis', 'Anemia', 'Atrial fibrillation', 'Boerhaave', 'Bronchiectasis', 
    'Bronchiolitis', 'Bronchitis', 'Bronchospasm / acute asthma exacerbation', 'Chagas', 
    'Chronic rhinosinusitis', 'Cluster headache', 'Croup', 'Ebola', 'Epiglottitis', 'GERD', 
    'Guillain-Barré syndrome', 'HIV (initial infection)', 'Influenza', 'Inguinal hernia', 
    'Larygospasm', 'Localized edema', 'Myasthenia gravis', 'Myocarditis', 'PSVT', 
    'Pancreatic neoplasm', 'Panic attack', 'Pericarditis', 'Pneumonia', 'Possible NSTEMI / STEMI', 
    'Pulmonary embolism', 'Pulmonary neoplasm', 'SLE', 'Sarcoidosis', 'Scombroid food poisoning', 
    'Spontaneous pneumothorax', 'Spontaneous rib fracture', 'Stable angina', 'Tuberculosis', 
    'URTI', 'Unstable angina', 'Viral pharyngitis', 'Whooping cough'
]

def predict_disease(text, top_k=3):
    print(f"Analyzing symptoms: '{text}'...")
    
    # 1. Exact Preprocessing (Must match our training phase!)
    encoding = tokenizer(
        text,
        add_special_tokens=True,
        max_length=128,
        padding='max_length',
        truncation=True,
        return_attention_mask=True,
        return_tensors='pt'
    )
    
    input_ids = encoding['input_ids'].to(device)
    attention_mask = encoding['attention_mask'].to(device)
    
    # 2. Make the Prediction
    with torch.no_grad():
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        probabilities = torch.sigmoid(outputs.logits)[0]
        
    # 3. Extract the Top K Guesses
    top_probs, top_indices = torch.topk(probabilities, top_k)
    
    # 4. Format the Results
    results = []
    for i in range(top_k):
        idx = top_indices[i].item()
        prob = top_probs[i].item()
        disease_name = DISEASES[idx]
        results.append((disease_name, prob * 100))
        
    return results

# --- LET'S TEST IT! ---
test_symptoms = "I am a 22-year-old male. I have a really bad sore throat, a runny nose, and I have been coughing all day. I feel a bit feverish too."

predictions = predict_disease(test_symptoms)

print("\n=== AI DIAGNOSIS ===")
for rank, (disease, confidence) in enumerate(predictions, 1):
    print(f"{rank}. {disease}: {confidence:.2f}%")

Analyzing symptoms: 'I am a 22-year-old male. I have a really bad sore throat, a runny nose, and I have been coughing all day. I feel a bit feverish too.'...

=== AI DIAGNOSIS ===
1. Allergic sinusitis: 99.97%
2. URTI: 0.03%
3. Bronchiolitis: 0.02%


In [7]:
test_cases = {
    "Anemia Case (Formatted)": "I am a 28-year-old female. I came into the clinic complaining of feeling incredibly exhausted lately. I get out of breath walking up the stairs, and my hands and feet are constantly freezing.",
    
    "GERD Case (Formatted)": "I am a 45-year-old male. I came into the clinic complaining of a terrible burning pain right in the middle of my chest after I eat dinner. It gets worse when I lie down, and I wake up with a sour, acidic taste in the back of my throat.",
    
    "Panic Attack Case (Formatted)": "I am a 30-year-old female. I came into the clinic complaining of my heart pounding out of my chest. I could not catch my breath, I was sweating profusely, and I felt like I was going to die."
}

print("=== TEMPLATE-MATCHED AI DIAGNOSIS ===\n")

for case_name, symptoms in test_cases.items():
    print(f"--- Analyzing: {case_name} ---")
    predictions = predict_disease(symptoms)
    for rank, (disease, confidence) in enumerate(predictions, 1):
        print(f"{rank}. {disease}: {confidence:.2f}%")
    print("\n")

=== TEMPLATE-MATCHED AI DIAGNOSIS ===

--- Analyzing: Anemia Case (Formatted) ---
Analyzing symptoms: 'I am a 28-year-old female. I came into the clinic complaining of feeling incredibly exhausted lately. I get out of breath walking up the stairs, and my hands and feet are constantly freezing.'...
1. PSVT: 71.32%
2. Anemia: 1.05%
3. Pulmonary neoplasm: 0.78%


--- Analyzing: GERD Case (Formatted) ---
Analyzing symptoms: 'I am a 45-year-old male. I came into the clinic complaining of a terrible burning pain right in the middle of my chest after I eat dinner. It gets worse when I lie down, and I wake up with a sour, acidic taste in the back of my throat.'...
1. Pneumonia: 98.01%
2. Panic attack: 0.79%
3. Bronchitis: 0.29%


--- Analyzing: Panic Attack Case (Formatted) ---
Analyzing symptoms: 'I am a 30-year-old female. I came into the clinic complaining of my heart pounding out of my chest. I could not catch my breath, I was sweating profusely, and I felt like I was going to die.'...
1. 

In [10]:
import pandas as pd
import random

# Load the local CSV file
df_test = pd.read_csv('translated_train_data.csv')

# Pick a random patient from the dataset
random_idx = random.randint(0, len(df_test) - 1)
ground_truth_disease = df_test.iloc[random_idx]['PATHOLOGY']
exact_text = df_test.iloc[random_idx]['NARRATIVE']

print(f"--- THE GROUND TRUTH TEST ---")
print(f"Actual Disease from CSV: **{ground_truth_disease}**\n")

# Run your exact predict function
predictions = predict_disease(exact_text)

print("\n=== AI DIAGNOSIS ===")
for rank, (disease, confidence) in enumerate(predictions, 1):
    print(f"{rank}. {disease}: {confidence:.2f}%")

--- THE GROUND TRUTH TEST ---
Actual Disease from CSV: **Scombroid food poisoning**

Analyzing symptoms: 'I am an 77-year-old male. I came into the clinic today because I have any lesions, redness or problems on your skin that you believe are related to the condition you are consulting for. Regarding the question 'What color is the rash', my answer is pink. No, I am not ir lesions peel off.. Regarding the question 'Is the rash swollen', my answer is 2. The affected region is located on my back of the neck, biceps(R), biceps(L), mouth, and ankle(R). Yes, regarding the question 'is the lesion (or are the lesions) larger than 1cm', my answer is. On a scale of 1 to 10, the severity of the itching is a 8. No, I am not traveling out of the country recently. To give you more context: Regarding the question 'Have you had diarrhea or an increase in stool frequency', my answer is I feel lightheaded and dizzy or do you feel like you are about to faint. Regarding the question 'Did your cheeks sudd